In [1]:

import ray
from ray import tune


## Using RLlib to train a standard environment

In [23]:
# #Tune basic
# from ray import tune

import time
# def objective(step, alpha, beta):
#     return (0.1 + alpha * step / 100)**(-1) + beta * 0.1


# def training_function(config):
#     # Hyperparameters
#     alpha, beta = config["alpha"], config["beta"]
#     for step in range(10):
#         # Iterative training function - can be any arbitrary training procedure.
#         intermediate_score = objective(step, alpha, beta)
#         # Feed the score back back to Tune.
#         tune.report(mean_loss=intermediate_score)


# analysis = tune.run(
#     training_function,
#     config={
#         "alpha": tune.grid_search([0.001, 0.01, 0.1]),
#         "beta": tune.choice([1, 2, 3,6])
#     })

# print("Best config: ", analysis.get_best_config(
#     metric="mean_loss", mode="min"))

# # Get a dataframe for analyzing trial results.
# df = analysis.results_df

from ray import tune
ray.shutdown()
ray.init(num_cpus=5,num_gpus=1)

# class Trainable(tune.Trainable):
#     def setup(self, config):
#         # config (dict): A dict of hyperparameters
#         self.x = 0
#         self.a = config["a"]
#         self.b = config["b"]

#     def step(self):  # This is called iteratively.
#         score = objective(self.x, self.a, self.b)
#         self.x += 1
#         time.sleep(1)
#         print("x is", self.x)
#         return {"score": score}

# analysis = tune.run(
#     Trainable,
#     num_samples=3,
#     stop={"training_iteration": 20},
#     config={
#         "a":tune.uniform(-5, -1),
#         "b": tune.grid_search([3,4,6,5,8])
#     })

# print('best config: ', analysis.get_best_config(metric="score", mode="max"))


2021-01-13 18:03:01,345	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.0.0.5',
 'raylet_ip_address': '10.0.0.5',
 'redis_address': '10.0.0.5:6379',
 'object_store_address': '/tmp/ray/session_2021-01-13_18-03-00_928457_9463/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-13_18-03-00_928457_9463/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-01-13_18-03-00_928457_9463',
 'metrics_export_port': 62877,
 'node_id': 'ee84c0baa268840d82da0c8b05bd53f5e55088ad2679b61caa8619c4'}

In [3]:
# ! rllib train --env=PongNoFrameskip-v4 --run=DQN --config '{"num_workers": 4,  "framework":"torch", "num_gpus":1}'


In [4]:
# from ray.rllib.agents.ppo import PPOTrainer
# from ray.rllib.agents.impala import ImpalaTrainer

# analysis = tune.run(PPOTrainer, config={"env": "CartPole-v0", "num_workers": 4,  "framework":"torch", "num_gpus":1,
#                                "lr":1e-3},
#                                                         stop={"training_iteration": 20},checkpoint_freq=3, checkpoint_at_end=True)  # "log_level": "INFO" for verbose,
#                                                      # "framework": "tfe"/"tf2" for eager,
#                                                      # "framework": "torch" for PyTorch
# # print('best config: ', analysis.get_best_config(metric="episode_reward_mean", mode="max"))


In [6]:
from ray.rllib.agents.ppo import PPOTrainer
trainer = PPOTrainer(env="CartPole-v0", config={"framework": "tfe", "num_workers": 0})

2021-01-12 03:44:58,134	INFO trainer.py:621 -- Executing eagerly, with eager_tracing=False
2021-01-12 03:44:58,136	INFO trainer.py:651 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-01-12 03:45:00,606	WARNING util.py:44 -- Install gputil for GPU system monitoring.


In [4]:
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean", mode="max"),
    metric="episode_reward_mean")
checkpoint_path=checkpoints[0][0]

In [8]:
checkpoint_path='/home/azureuser/ray_results/PPO_2021-01-12_03-33-56/PPO_CartPole-v0_00533_00000_0_2021-01-12_03-33-56/checkpoint_3/checkpoint-3'

In [7]:
# pip install https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-1.2.0.dev0-cp36-cp36m-manylinux2014_x86_64.whl

In [13]:
config={"env": "CartPole-v0", "num_workers": 4,  "num_gpus":1,"framework": "torch"}
agent = PPOTrainer(config=config)
agent.restore(checkpoint_path)


(pid=21661) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=21661) Instructions for updating:
(pid=21661) non-resource variables are not supported in the long term
2021-01-12 03:47:53,606	WARNING util.py:44 -- Install gputil for GPU system monitoring.
2021-01-12 03:47:53,616	INFO trainable.py:329 -- Restored on 10.0.0.5 from checkpoint: /home/azureuser/ray_results/PPO_2021-01-12_03-33-56/PPO_CartPole-v0_00533_00000_0_2021-01-12_03-33-56/checkpoint_3/checkpoint-3
2021-01-12 03:47:53,617	INFO trainable.py:336 -- Current state after restoring: {'_iteration': 3, '_timesteps_total': None, '_time_total': 24.149604082107544, '_episodes_total': 291}


In [14]:
# instantiate env class
import gym

env = gym.make("CartPole-v0")

# run until episode ends
episode_reward = 0
done = False
obs = env.reset()
while not done:
    action = agent.compute_action(obs)
    print(action)
    obs, reward, done, info = env.step(action)
    episode_reward += reward


0
0
1
1
0
1
1
0
1
0
1
0
0
1
0
1
0
0
1
0
0
1
1
1
1
0
0
1
1
1
0
0
1
0
0
1
0
1
1
0
1
1
0
1
0
1
0
1
1
0
0
1
0
1
0
0
0
1
1
0
0
0
1
1
0
1
1
1
1
0
0
0
1
0
1
0
1
0
1
1
1
0
0
0
1
0
0
0
0
1
1
1
1
1
0
1
1
0
1
0
0
0
0
1
1
0
1
1
1
0
1
0
0
0
0
1
0
1
1
1
0
0
1
1
0
0
0
1
1
0
0
1
1
0
1
1
0
0
1
1
0
0
0
1
1
1
1
0
0
0
0
1
1
1
0
1
0
0
1
0
1
1
0
0
0
1
1
1
0
0
0
0
0
0
0
1
1
1
1
1
1
1
0
1
0
1
1
0
1
1
0
0
0
1
1
0
0
0
1
1


2021-01-12 03:48:08,415	WARNING worker.py:1039 -- The actor or task with ID ffffffffffffffff65a1a3aaa614cba3c58a358701000000 cannot be scheduled right now. It requires {CPU: 1.000000} for placement, but this node only has remaining {0.000000/5.000000 CPU, 66.455078 GiB/66.455078 GiB memory, 1.000000/1.000000 GPU, 1.000000/1.000000 node:10.0.0.5, 22.412109 GiB/22.412109 GiB object_store_memory, 1.000000/1.000000 accelerator_type:P100}
. In total there are 0 pending tasks and 3 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.


In [15]:
from ray.rllib.models.preprocessors import get_preprocessor
prep = get_preprocessor(env.observation_space)(env.observation_space)
prep.transform(env.reset()).shape

(84, 84, 3)

In [10]:
from ray.rllib.agents.ppo import PPOTrainer
import numpy as np
trainer = PPOTrainer(env="CartPole-v0", config={ "num_workers": 0})
policy = trainer.get_policy()
logits, _ = policy.model.from_batch({"obs": np.array([[0.1, 0.2, 0.3, 0.4]])})

2021-01-12 03:39:52,844	WARNING util.py:43 -- Install gputil for GPU system monitoring.


## Customizing RLlib to implement Connect4

In [2]:
from ray.rllib.contrib.alpha_zero.models.custom_torch_models import ConvNetModel
from ray.rllib.contrib.alpha_zero.environments.cartpole import CartPole
from ray.rllib.models.catalog import ModelCatalog
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer

Instructions for updating:
non-resource variables are not supported in the long term


### Custom environment with connect 4

In [3]:
import gym, ray
from gym.spaces import Discrete, Box, Tuple, Dict
from copy import deepcopy

from kaggle_environments import make
import numpy as np


class MyEnv(gym.Env):
    def __init__(self, env_config):
        #trainer environment with a random opponent
        self.env_org = make("connectx", debug=False)
        self.trainer = self.env_org.train([None, "random"])
        self.current_player=1
        self.action_space = Discrete(7)
#         self.observation_space = Box(low=0,high=2, shape=(1,6,7), dtype=np.uint8)
        
        self.observation_space = Dict({
            "obs": Box(low=0,high=2, shape=(1,6,7), dtype=np.uint8),
            "action_mask": Box(low=0, high=1, shape=(self.action_space.n, ))
        })
        self.running_reward = 0

      
    def reset(self):
        board = self.env_org.reset()[0]['observation']['board']
        self.trainer.reset()
        self.running_reward = 0
        board = np.array(board).reshape(1,6,7)
        board[0,:,:]=self.current_player

        return {"obs":board, "action_mask":np.array([1,1,1,1,1,1,1])}
    def step(self, action):
        obs, reward, done, info = self.trainer.step(int(action))

        board =obs['board']
        if reward == None:
            reward = -1
        self.running_reward += reward
        board = np.array(board).reshape(1,6,7)
        board[0,:,:]=self.current_player
        info ={"info":"empty"}
        return {"obs":board, "action_mask":np.array([1,1,1,1,1,1,1])}, reward, done, info
    
    
    def set_state(self, state):
        self.running_reward = state[1]
        self.env_org = deepcopy(state[0])
        self.trainer = self.env_org.train([None, "random"])

        board =self.env_org.state[0]['observation']['board']
        board = np.array(board).reshape(1,6,7)
        board[0,:,:]=self.current_player
        return {"obs": board, "action_mask": np.array([1,1,1,1,1,1,1])}

    def get_state(self):
        return deepcopy(self.env_org), self.running_reward

Loading environment football failed: No module named 'gfootball'


### Custom NN Model

In [7]:
from abc import ABC
import numpy as np

from ray.rllib.models.modelv2 import restore_original_dimensions
from ray.rllib.models.preprocessors import get_preprocessor
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.framework import try_import_torch

torch, nn = try_import_torch()


def convert_to_tensor(arr):
    tensor = torch.from_numpy(np.asarray(arr))
    if tensor.dtype == torch.double:
        tensor = tensor.float()
    return tensor


class ActorCriticModel(TorchModelV2, nn.Module, ABC):
    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)

        self.preprocessor = get_preprocessor(obs_space.original_space)(
            obs_space.original_space)

        self.shared_layers = None
        self.actor_layers = None
        self.critic_layers = None

        self._value_out = None

    def forward(self, input_dict, state, seq_lens):
        x = input_dict["obs"]
        x = self.shared_layers(x)
        # actor outputs
        logits = self.actor_layers(x)

        # compute value
        self._value_out = self.critic_layers(x)
        return logits, None

    def value_function(self):
        return self._value_out

    def compute_priors_and_value(self, obs):
        obs = convert_to_tensor([self.preprocessor.transform(obs)])
        input_dict = restore_original_dimensions(obs, self.obs_space, "torch")

        with torch.no_grad():
            model_out = self.forward(input_dict, None, [1])
            logits, _ = model_out
            value = self.value_function()
            logits, value = torch.squeeze(logits), torch.squeeze(value)
            priors = nn.Softmax(dim=-1)(logits)

            priors = priors.cpu().numpy()
            value = value.cpu().numpy()

            return priors, value


class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


class ConvNetModel(ActorCriticModel):
    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        ActorCriticModel.__init__(self, obs_space, action_space, num_outputs,
                                  model_config, name)

        in_channels = model_config["custom_model_config"]["in_channels"]
        feature_dim = model_config["custom_model_config"]["feature_dim"]

        self.shared_layers = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=2, stride=1),
            nn.Conv2d(32, 64, kernel_size=2, stride=1),
            nn.Conv2d(64, 64, kernel_size=2, stride=1), Flatten(),
            nn.Linear(768, feature_dim))

        self.actor_layers = nn.Sequential(
            nn.Linear(in_features=feature_dim, out_features=action_space.n))

        self.critic_layers = nn.Sequential(
            nn.Linear(in_features=feature_dim, out_features=1))

        self._value_out = None

In [9]:
ray.shutdown()
ray.init(num_cpus=5,num_gpus=1)

ModelCatalog.register_custom_model("ConvNetModel", ConvNetModel)



config={
    "env": MyEnv,
    "env_config": {},
    "num_workers": 4,
#         "num_gpus":1,
    "framework":"torch",

    "rollout_fragment_length": 20,
    "train_batch_size": 80,
    "sgd_minibatch_size": 64,
    "lr": 1e-4,
    "num_sgd_iter": 1,
    "mcts_config": {
        "puct_coefficient": 1.5,
        "num_simulations": 100,
        "temperature": 1.0,
        "dirichlet_epsilon": 0.20,
        "dirichlet_noise": 0.03,
        "argmax_tree_policy": False,
        "add_dirichlet_noise": True,
    },
    "ranked_rewards": {
        "enable": True,
    },
    "model": {
        "custom_model": "ConvNetModel",
        "custom_model_config":{"in_channels":1, "feature_dim":64}
    },
}
alphago_trainer = AlphaZeroTrainer(config= config)

analysis = tune.run(
   AlphaZeroTrainer
    ,
    config=config,
    max_failures=0,
    stop={"episode_reward_mean": -0.2,"time_total_s": 3600},
    checkpoint_freq=2,
    local_dir='./logs_alphago')

2021-01-13 19:15:02,220	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=6347) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=6347) Instructions for updating:
(pid=6347) non-resource variables are not supported in the long term
(pid=6347) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=6347) Instructions for updating:
(pid=6347) non-resource variables are not supported in the long term
(pid=6347) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tenso

(pid=6349) Loading environment football failed: No module named 'gfootball'
(pid=6349) Loading environment football failed: No module named 'gfootball'
(pid=6349) Loading environment football failed: No module named 'gfootball'
(pid=6349) Loading environment football failed: No module named 'gfootball'
(pid=6347) Loading environment football failed: No module named 'gfootball'
(pid=6347) Loading environment football failed: No module named 'gfootball'
(pid=6347) Loading environment football failed: No module named 'gfootball'
(pid=6347) Loading environment football failed: No module named 'gfootball'
(pid=6348) Loading environment football failed: No module named 'gfootball'
(pid=6348) Loading environment football failed: No module named 'gfootball'
(pid=6348) Loading environment football failed: No module named 'gfootball'
(pid=6348) Loading environment football failed: No module named 'gfootball'
(pid=6345) Loading environment football failed: No module named 'gfootball'
(pid=6345) L

(pid=6347) 2021-01-13 19:15:06,462	WARNING catalog.py:465 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
(pid=6347) 2021-01-13 19:15:06,462	WARNING catalog.py:465 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
(pid=6347) 2021-01-13 19:15:06,462	WARNING catalog.py:465 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
(pid=6347) 2021-01-13 19:15:06,462	WARNING catalog.py:465 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
(pid=6345) 2021-01-13 19:15:06,465	WARNING catalog.py:465 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
(pid=6345) 2021-01-13 19:15:06,465	WARNING catalog.py:465 -- Custom ModelV2 should accept 

Trial name,status,loc
AlphaZero_MyEnv_ab13c_00000,RUNNING,


(pid=6346) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=6346) Instructions for updating:
(pid=6346) non-resource variables are not supported in the long term
(pid=6346) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=6346) Instructions for updating:
(pid=6346) non-resource variables are not supported in the long term
(pid=6346) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=6346

(pid=6346) Loading environment football failed: No module named 'gfootball'
(pid=6346) Loading environment football failed: No module named 'gfootball'
(pid=6346) Loading environment football failed: No module named 'gfootball'
(pid=6346) Loading environment football failed: No module named 'gfootball'


(pid=6346) 2021-01-13 19:15:18,973	INFO trainer.py:651 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=6346) 2021-01-13 19:15:18,973	INFO trainer.py:651 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=6346) 2021-01-13 19:15:18,973	INFO trainer.py:651 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=6346) 2021-01-13 19:15:18,973	INFO trainer.py:651 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


KeyboardInterrupt: 

In [ ]:
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean", mode="max"),
    metric="episode_reward_mean")
checkpoint_path=checkpoints[0][0]
print(checkpoint_path)
alphago_trainer.restore(checkpoint_path)

In [ ]:
from kaggle_environments import make

env = make("connectx", debug=True)

# Training agent in first position (player 1) against the default random agent.
trainer = env.train([None, "random"])
done=False
obs = trainer.reset()

while (not done):
    board = obs['board']
    player= obs['mark']
#     print(player)
    board = np.array(board).reshape(1,6,7)
    board[0,:,:]=player
    action = int(alphago_trainer.compute_action(board)) # Action for the agent being trained.
    obs, reward, done, info = trainer.step(action)

    if reward == None:
        print("none catched, obs is ", obs)
        
#     print("info", info)

    if done:
        print(reward)
#         print(obs)
        env.render(mode="ipython", width=500, height=450)

        obs = trainer.reset()


In [24]:
ray.shutdown()
ray.init(num_cpus=5,num_gpus=1)

from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel
from ray.rllib.contrib.alpha_zero.environments.cartpole import CartPole
from ray.rllib.models.catalog import ModelCatalog


ModelCatalog.register_custom_model("dense_model", DenseModel)

tune.run(
    "contrib/AlphaZero",
    stop={"training_iteration": 20},
    max_failures=0,
    config={
        "env": CartPole,
        "num_workers": 4,
                "num_gpus":1,

        "rollout_fragment_length": 50,
        "train_batch_size": 500,
        "sgd_minibatch_size": 64,
        "lr": 1e-4,
        "num_sgd_iter": 1,
        "mcts_config": {
            "puct_coefficient": 1.5,
            "num_simulations": 100,
            "temperature": 1.0,
            "dirichlet_epsilon": 0.20,
            "dirichlet_noise": 0.03,
            "argmax_tree_policy": False,
            "add_dirichlet_noise": True,
        },
        "ranked_rewards": {
            "enable": True,
        },
        "model": {
            "custom_model": "dense_model",
        },
    },
)


2021-01-13 18:03:24,372	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


Trial name,status,loc
contrib_AlphaZero_CartPole_a2887_00000,RUNNING,


(pid=19665) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=19665) Instructions for updating:
(pid=19665) non-resource variables are not supported in the long term
(pid=19665) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=19665) Instructions for updating:
(pid=19665) non-resource variables are not supported in the long term
(pid=19665) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(p

Result for contrib_AlphaZero_CartPole_a2887_00000:
  {}
  


Trial name,status,loc
contrib_AlphaZero_CartPole_a2887_00000,ERROR,
Trial name,# failures,error file
contrib_AlphaZero_CartPole_a2887_00000,1,/home/azureuser/ray_results/contrib/AlphaZero/contrib_AlphaZero_CartPole_a2887_00000_0_2021-01-13_18-03-26/error.txt


Trial name,status,loc
contrib_AlphaZero_CartPole_a2887_00000,ERROR,
Trial name,# failures,error file
contrib_AlphaZero_CartPole_a2887_00000,1,/home/azureuser/ray_results/contrib/AlphaZero/contrib_AlphaZero_CartPole_a2887_00000_0_2021-01-13_18-03-26/error.txt


TuneError: ('Trials did not complete', [contrib_AlphaZero_CartPole_a2887_00000])

In [ ]:
import gym, ray
from gym.spaces import Discrete, Box, Tuple, Dict
from copy import deepcopy

from kaggle_environments import make
import numpy as np
class MyEnv2(gym.Env):
    def __init__(self, env_config):
        #trainer environment with a random opponent
        self.env = make("connectx", debug=False)
        self.current_player=1
        self.action_space = Discrete(7)
        self.observation_space = Box(low=0,high=2, shape=(1,6,7), dtype=np.uint8)
        

      
    def reset(self):
        obs,_ =self.env.reset()
        board =obs['observation']['board']
        board = np.array(board).reshape(1,6,7)
        self.current_player=obs['observation']['mark']
        board[0,:,:]=self.current_player
        return board

    def step(self, action):
        if self.current_player==1:
            obs =self.env.step([int(action), None])[0]

            self.current_player=2
        else:
            obs =self.env.step([None, int(action)])[0]

            self.current_player=1

        board =obs['observation']['board']
        reward = obs['reward']
        done =self.env.done
        if reward == None:
            reward = -1
        board = np.array(board).reshape(1,6,7)
        board[0,:,:]=self.current_player
        info ={"info":"empty"}
        return board, reward, done, info
    
    

In [2]:
import gym, ray
from gym.spaces import Discrete, Box, Tuple, Dict
from copy import deepcopy

from kaggle_environments import make
import numpy as np
class MyEnv3(gym.Env):
    def __init__(self, env_config):
        #trainer environment with a random opponent
        self.env = make("connectx", debug=False).train([None, "negamax"])
        self.current_player=1
        self.action_space = Discrete(7)
        self.observation_space = Box(low=0,high=2, shape=(1,6,7), dtype=np.uint8)
        

      
    def reset(self):
        obs =self.env.reset()
        board =obs['board']
        board = np.array(board).reshape(1,6,7)
        self.current_player=obs['mark']
        board[0,:,:]=self.current_player
        return board

    def step(self, action):
        
        obs, reward, done, info = self.env.step(int(action))
#         if self.current_player==1:
#             self.current_player=2
#         else:
#             self.current_player=1

        board =obs['board']
        if reward == None:
            reward = -1
        board = np.array(board).reshape(1,6,7)
        board[0,:,:]=self.current_player
        info ={"info":"empty"}
        return board, reward, done, info
    
    

Loading environment football failed: No module named 'gfootball'


In [22]:
# env_config={}
# def env_creator(env_config):
#     return MyEnv2(env_config)  # return an env instance

# register_env("MyEnv2", env_creator)

In [192]:
# !rllib train --run A3C --checkpoint-freq 1 --stop '{"episode_reward_mean": 0.96,"time_total_s": 3600}' --env=CartPole-v0 --config '{"framework":"torch","num_gpus": 2,"num_workers": 11,"train_batch_size": 1000, "model": {"conv_filters": [[1, [1,1], 64], [64, [1,1], 12]]}}'

In [216]:
!rm -r ./logs3/A3C

rm: cannot remove './logs3/A3C/A3C_0_2020-09-07_21-59-34f9a3aa01': Directory not empty
rm: cannot remove './logs3/A3C/A3C_0_2020-09-07_22-00-509a64f858': Directory not empty


In [3]:
ray.shutdown()
ray.init(num_cpus=6,num_gpus=1)
tune.run(
    run_or_experiment="PPO",
    config={
        "framework":"torch",
        
        "env": MyEnv3,
        "num_gpus": 1,
        "num_workers": 5,
        "train_batch_size": 1000,
        "model": {"conv_filters": [[1, [1,1], 64], [64, [1,1], 12]]}

    },
    stop={"episode_reward_mean": 0.8,"time_total_s": 3600},
    checkpoint_freq=100,
    local_dir='./logs4')

In [11]:
ray.shutdown()

In [12]:
ray.shutdown()
ray.init(num_gpus=1)


2021-01-02 22:21:26,514	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.0.0.5',
 'raylet_ip_address': '10.0.0.5',
 'redis_address': '10.0.0.5:6379',
 'object_store_address': '/tmp/ray/session_2021-01-02_22-21-26_146423_15904/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-02_22-21-26_146423_15904/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-01-02_22-21-26_146423_15904',
 'metrics_export_port': 63002,
 'node_id': '94f50afe00179c8e7936f25a8e3546903b96f933'}

In [95]:
!ls ~/ray_results/


In [147]:
from ray.rllib.agents.a3c import A3CTrainer
class MyTrainer(A3CTrainer):
    def __init__(self, env, config):
        
        super().__init__(env=env, config=config)
    def export_model(self,format, dir):
        pass

In [221]:
from ray.rllib.agents.a3c import A3CTrainer
config={
    "framework":"torch",

    "env": MyEnv2,
    "num_gpus": 2,
    "num_workers": 1,
    "train_batch_size": 1000,
    "model": {"conv_filters": [[1, [1,1], 64], [64, [1,1], 12]]}

}
a3ctrainer =MyTrainer(env=MyEnv2, config=config)
a3ctrainer.restore(checkpoint_path="/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpunc12/code/ray/logs4/A3C/A3C_0_2020-09-07_22-20-11273fd95a/checkpoint_700/checkpoint-700")

2020-09-08 00:41:13,238	WARNING a3c.py:59 -- `sample_async=True` is not supported for PyTorch! Multithreading can lead to crashes.
2020-09-08 00:41:13,284	WARNING util.py:39 -- Install gputil for GPU system monitoring.
2020-09-08 00:41:13,490	INFO trainable.py:482 -- Restored on 10.0.0.5 from checkpoint: /mnt/batch/tasks/shared/LS_root/mounts/clusters/gpunc12/code/ray/logs4/A3C/A3C_0_2020-09-07_22-20-11273fd95a/checkpoint_700/checkpoint-700
2020-09-08 00:41:13,491	INFO trainable.py:489 -- Current state after restoring: {'_iteration': 700, '_timesteps_total': None, '_time_total': 3478.3344514369965, '_episodes_total': 802272}
(pid=123141) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:88: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=123141) Instructions for updating:
(pid=123141) non-resource variables are not supported in the 

In [16]:
torch.cuda.is_available()

True

In [227]:
env = MyEnv2(env_config={})
obs =env.reset()
a3ctrainer.compute_action(obs)

3

In [263]:
from kaggle_environments import make

env = make("connectx", debug=True)

# Training agent in first position (player 1) against the default random agent.
trainer = env.train([None, "random"])
done=False
obs = trainer.reset()

while (not done):
    board = obs['board']
    player= obs['mark']
#     print(player)
    board = np.array(board).reshape(1,6,7)
    board[0,:,:]=player
    action = int(a3ctrainer.compute_action(board)) # Action for the agent being trained.
    obs, reward, done, info = trainer.step(action)

    if reward == None:
        print("none catched, obs is ", obs)
        
#     print("info", info)

    if done:
        print(reward)
#         print(obs)
        env.render(mode="ipython", width=500, height=450)

        obs = trainer.reset()


-1


In [226]:
from kaggle_environments import make

env = make("connectx", debug=True)

# Training agent in first position (player 1) against the default random agent.
trainer = env.train([None, "random"])

obs = trainer.reset()
while (not done):
    board = obs['board']
    player= obs['mark']
#     print(player)
    board = np.array(board).reshape(1,6,7)
    board[0,:,:]=player
    action = int(a3ctrainer.compute_action(board)) # Action for the agent being trained.
    obs, reward, done, info = trainer.step(action)

    if reward == None:
        print("none catched, obs is ", obs)
        
#     print("info", info)

    if done:
        print(reward)
#         print(obs)
        env.render(mode="ipython", width=500, height=450)

        obs = trainer.reset()
